In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
filepath = "C:\\Users\\Vidhish17\\Desktop\\20Credit\\data\\paris-lille-10\\training_10_classes\\Paris_features_clean.csv"
# filepath = "C:\\Users\\Vidhish17\\Desktop\\20Credit\\data\\vaihingen\\features_010_jakteristics_14.csv"

In [3]:
data = pd.read_csv(filepath)

In [4]:
# data.describe()

In [5]:
data.drop(columns=['Unnamed: 0'], inplace=True)
print(len(data))

38311606


In [6]:
# data.drop(columns=['level_0'], inplace=True)
# data.head()


In [7]:
data = data.astype(float)
# grouped = data.groupby('class')
# averages = grouped.mean()
# variances = grouped.var()
# print(averages)
# print(variances)

In [8]:
def compute_covariance_matrix(data, regularization=0):
    cov_matrix = np.cov(data, rowvar=False)
    cov_matrix += regularization * np.eye(cov_matrix.shape[0]) #Helps in computing Gaussian PDF
    return cov_matrix

In [9]:
def fit(x_train, y_train):
    y_train = y_train.ravel()
    m = y_train.shape[0] 
    x_train = x_train.reshape(m, -1)
    input_feature = x_train.shape[1]
    
    class_label = 10 #ParisLille
    # class_label = 9 #Vaihingen

    """
    mu    : (μ) Denotes the population mean (Class wise mean feature vector)
    phi   : (Φ) Prior probability of each class
    sigma : (Σ) Covariance matrix of each class
    """
    
    mu = np.zeros((class_label, input_feature))                    # Mean vectors per class
    sigma = np.zeros((class_label, input_feature, input_feature))  # Covariance matrices per class
    phi = np.zeros(class_label)                                    # Prior probability per class

    for label in range(class_label):
        indices = (y_train == label)
        phi[label] = float(np.sum(indices)) / m
        mu[label] = np.mean(x_train[indices, :], axis=0)
        sigma[label] = compute_covariance_matrix(x_train[indices, :])
    
    return phi, mu, sigma

In [10]:
# has_nan = data.isnull().values.any()
# print(has_nan)

# data.columns

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.05, random_state=42)
print(f"Train shape: {train_data.shape}, Test shape: {test_data.shape}")

Train shape: (36396025, 18), Test shape: (1915581, 18)


In [12]:
import gc
del data
gc.collect()

0

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [14]:
# IGNORE ---
columns_to_scale = ['eigenvalue_sum', 'omnivariance', 'eigenentropy',
       'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2',
       'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz']

scaled_df = pd.DataFrame(scaler.fit_transform(train_data[columns_to_scale]), columns=columns_to_scale)
train_data[columns_to_scale] = scaled_df

In [15]:
train_data.head()

,x,y,z,class,eigenvalue_sum,omnivariance,eigenentropy,anisotropy,planarity,linearity,PCA1,PCA2,surface_variation,sphericity,verticality,nx,ny,nz
13294822,37.564068,-609.269531,42.262688,1.0,0.478073,0.454931,0.607656,0.797671,0.695600,0.112804,0.213193,0.853866,0.284086,0.202328,0.997223,0.277860,0.052058,0.498611
24026624,46.484673,-482.015381,46.835037,9.0,0.494393,0.187658,0.593016,0.990325,0.866648,0.125883,0.289351,0.929185,0.015003,0.009675,0.781721,0.987820,0.489049,0.390860
28638846,31.158279,-426.904816,41.635662,2.0,0.521197,0.210973,0.618406,0.987732,0.951354,0.038875,0.252850,0.975348,0.018163,0.012268,0.001330,0.474932,0.493991,0.000665
10963553,32.272594,-647.179565,43.080273,1.0,0.352958,0.329395,0.476808,0.826290,0.518905,0.316440,0.304180,0.740008,0.274352,0.173710,0.300925,0.146247,0.551790,0.150462
8374908,18.735130,-679.694580,54.048481,2.0,0.640977,0.584990,0.749916,0.836177,0.270806,0.573467,0.442140,0.539460,0.302276,0.163823,0.497903,0.485139,0.932150,0.751048


In [16]:
len(train_data)

36396025

In [17]:
del scaled_df
gc.collect()

0

In [18]:
nan_rows_count = train_data.isna().any(axis=1).sum()
print(f"Number of rows with NaN values in train_data: {nan_rows_count}")

Number of rows with NaN values in train_data: 1819961


In [19]:
del nan_rows_count
gc.collect()

0

In [20]:
train_data.dropna(inplace=True)
len(train_data)

34576064

In [21]:
x_train = train_data[['x', 'y', 'z', 'eigenvalue_sum', 'omnivariance', 'eigenentropy',
       'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2',
       'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz']]
y_train = train_data[['class']]

x_train_values = x_train.values
y_train_values = y_train.values

In [22]:
x_test = test_data[['x', 'y', 'z', 'eigenvalue_sum', 'omnivariance', 'eigenentropy',
       'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2',
       'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz']]

y_test = test_data[['class']]

x_test_values = x_test.values
y_test_values = y_test.values

In [23]:
%%time
phi, mu, sigma = fit(x_train_values, y_train_values)

CPU times: total: 14.1 s
Wall time: 14.8 s


In [24]:
# print(phi)
# print(mu)
# print(sigma)

In [25]:
import math
def multivariate_gaussian_pdf(x, mean, cov):
    d = mean.shape[0] #dimensionality of the input
    exponent = -0.5 * np.dot(np.dot((x - mean).T, np.linalg.inv(cov)), (x - mean)) # -(1/2) . Transpose(x−μ) . Inverse(Σ or Covariance Matx) . (x−μ)
    prefactor = 1 / np.sqrt(((2 * np.pi) ** d )*(np.linalg.det(cov))) # 1 / Sqrt( (2π)^d . |Σ| ) 
    return np.exp(exponent)*prefactor

In [26]:
# Covariance matrices must be positive semidefinite because variance (and covariance) can't be negative
def is_positive_semidefinite(matrix):
    eigenvalues, _ = np.linalg.eig(matrix)
    print(eigenvalues)
    return np.all(eigenvalues >= 0)

matrix = sigma[1]
# print(matrix)
positive_semidefinite = is_positive_semidefinite(matrix)
if positive_semidefinite:
    print("The matrix is positive semidefinite.")
else:
    print("The matrix is not positive semidefinite.")

[1.76298474e+04 3.96826539e+01 3.05228071e-01 1.46178846e-01
 1.29617986e-01 6.23128984e-02 5.03654825e-02 2.09103316e-02
 1.10043321e-02 7.03154481e-03 1.40306770e-03 7.19533447e-04
 2.02609662e-04 6.14229663e-06 2.36524520e-15 1.06241779e-16
 8.83663149e-16]
The matrix is positive semidefinite.


In [27]:
def give_epistemic(X_test):
    x_test = X_test.values
    feature_densities = []
    for i in range (x_test.shape[0]):
        rel_probs = {}
        # deno = 0.00000005
        deno = 0
        
        #for label in (1,2,5,8):
        for label in (0,1,2,3,4,5,6,7,8,9): #ParisLille
        # for label in (1, 2, 5, 8): #Vaihingen
            x = multivariate_gaussian_pdf(x_test[i], mu[label], sigma[label])
            deno += x
            rel_probs[label] = x
        # probs = [x/deno for x in rel_probs]
        probs = {}
        for k, v in rel_probs.items():
            probs[k] = v/deno
        feature_density = 0
        
        #labels = [1,2,5,8]
        labels = [0,1,2,3,4,5,6,7,8,9] #ParisLille
        # labels = [1, 2, 5, 8] #Vaihingen
        for j in range (len(labels)):
        # for j in labels:
            feature_density += phi[labels[j]]*probs[j]
        feature_densities.append([x_test[i], feature_density])
    
    epistemic_uncertainty = []
    for i in feature_densities:
        epistemic_uncertainty.append(1-i[1])
    return epistemic_uncertainty

In [28]:
#--------------------------------------------------------------------------------------------<Compute aleatoric uncertnty>---||
def get_aleatoric(X_test, softmax_probs):
    entropies = []
    for i in range (len(softmax_probs)):
        for j in softmax_probs[i]:
            entropy = 0
            if (j == 0):
                continue
            else:
                entropy+= -j*np.log(j)
        entropies.append(entropy)
    return entropies

In [29]:
%%time

X_epistemic = give_epistemic(x_train)

CPU times: total: 1h 52min 5s
Wall time: 2h 17min 37s


In [30]:
x_train['epistemic'] = X_epistemic
x_train['class'] = y_train_values

In [31]:
x_train.to_csv('./withEpistemic/x_train-paris-lille.csv', index=False)

In [32]:
X_test_epistemic = give_epistemic(x_test)
x_test['epistemic'] = X_test_epistemic
x_test['class'] = y_test_values
x_test.to_csv('./withEpistemic/x_test-paris-lille.csv', index=False)

In [33]:
# label_values = data['label']
# data.drop(columns=['label'], inplace=True)

In [34]:
# # Compute uncertainty for the entire dataset (without splitting into train and test)
# X_epistemic_full = give_epistemic(data)
# data['epistemic'] = X_epistemic_full
# data['label'] = label_values
# data.to_csv('./withEpistemic/full_data_with_uncertainty.csv', index=False)

In [35]:
phi

array([0.02585546, 0.46946547, 0.23779916, 0.0048461 , 0.00066792,
       0.00074245, 0.05940396, 0.00372295, 0.03566875, 0.16182779])

In [36]:
X_epistemic

[np.float64(0.8591260647128631),
 np.float64(0.9628752844695859),
 np.float64(0.9699667199808787),
 np.float64(0.7841629573451581),
 np.float64(0.7634860182548823),
 np.float64(0.7745613310663075),
 np.float64(0.8379452990063598),
 np.float64(0.7923388888586494),
 np.float64(0.9076515569313806),
 np.float64(0.7004435921359509),
 np.float64(0.9736554724816272),
 np.float64(0.7230856353197552),
 np.float64(0.8754749946244057),
 np.float64(0.7846939570136255),
 np.float64(0.85827412283652),
 np.float64(0.7624258535428111),
 np.float64(0.7903768334559148),
 np.float64(0.7201256553963818),
 np.float64(0.9268189093429279),
 np.float64(0.7650064871314439),
 np.float64(0.7215425821287402),
 np.float64(0.7371651730521249),
 np.float64(0.9782107688322784),
 np.float64(0.9401391060667426),
 np.float64(0.7629469720341165),
 np.float64(0.9187341447639973),
 np.float64(0.9331786165221068),
 np.float64(0.9662960530309606),
 np.float64(0.9459673282795994),
 np.float64(0.8659001328773158),
 np.float64(